# **`Product Recommendation by Association Rules Analytics`**

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.set_option('display.max_colwidth', 0)
print(f'pandas version : {pd.__version__}')

pandas version : 1.0.3


In [2]:
output_path = 'output/'

df = pd.read_csv(f'{output_path}final_allsr_2019.csv',
                 parse_dates=['INV_DATE'],
                 header=0)
df

,SHOWROOM,INV_DATE,INV_NO,ITEM_CODE,ITEM_NAME,QTY,PRICE,CUSTOMER,SR_CODE,INV_TYPE,...,M12,INV_WEEK_DAY,WD0,WD1,WD2,WD3,WD4,WD5,WD6,INV_MONTH_DAY
0,ONLINE,2019-09-23,CS-1256209/057,IG0681,"Callebaut Dark Chocolate Couverture 57.7% , 2.5kg",1.0,850.0,P13820,0,CS,...,0,0,0,0,0,0,0,0,0,23
1,ONLINE,2019-09-23,CS-1256209/057,IG0682,"Callebaut White Chocolate Couverture 28% ,2.5 kg",1.0,950.0,P13820,0,CS,...,0,0,0,0,0,0,0,0,0,23
2,ONLINE,2019-09-23,CS-1256209/057,PK0645,SYN Triangle Paper Gold Card 7x11 cm. (P/100) สามเหลี่ยม,1.0,90.0,P13820,0,CS,...,0,0,0,0,0,0,0,0,0,23
3,ONLINE,2019-09-23,CS-1256209/057,PK0405-8,SNY Round Paper Card 8 cm. P/100,1.0,90.0,P13820,0,CS,...,0,0,0,0,0,0,0,0,0,23
4,ONLINE,2019-09-23,CS-1256209/058,PK0115,CH32 Jelly/Mousse Cups Cap:130ml. dia.98x47x45H,200.0,3.8,R0408,0,CS,...,0,0,0,0,0,0,0,0,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36864,SR05,2019-12-30,KW256212/00299,797173100,Triangle 797173100 Stirring Spoo Nylon Black,1.0,550.0,P14486,2,KW,...,12,0,0,0,0,0,0,0,0,30
36865,SR05,2019-12-30,KW256212/00299,MA0362-CA,KitchenAid 5KSM7580XCCA Artisan Bowl 7Qt Candy Apple Red,1.0,39000.0,P14486,2,KW,...,12,0,0,0,0,0,0,0,0,30
36866,SR05,2019-12-30,KW256212/00299,018436002406,CDN POT750X High Heat Oven Thermometer-NSF (B502),1.0,410.0,P14486,2,KW,...,12,0,0,0,0,0,0,0,0,30
36867,SR05,2019-12-30,KW256212/00300,MA2204,Piron PF5004M Multi-Function Mecc Oven 4 Trays ECO (Caboto),1.0,42693.0,C2729,2,KW,...,12,0,0,0,0,0,0,0,0,30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36869 entries, 0 to 36868
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SHOWROOM       36869 non-null  object        
 1   INV_DATE       36869 non-null  datetime64[ns]
 2   INV_NO         36869 non-null  object        
 3   ITEM_CODE      36869 non-null  object        
 4   ITEM_NAME      36869 non-null  object        
 5   QTY            36869 non-null  float64       
 6   PRICE          36869 non-null  float64       
 7   CUSTOMER       36869 non-null  object        
 8   SR_CODE        36869 non-null  int64         
 9   INV_TYPE       36869 non-null  object        
 10  SELL_TYPE      36869 non-null  object        
 11  ITEM_TYPE      36869 non-null  object        
 12  CUSTOMER_TYPE  36869 non-null  object        
 13  INV_QUARTER    36869 non-null  int64         
 14  Q1             36869 non-null  int64         
 15  Q2             3686

In [4]:
df.columns

Index(['SHOWROOM', 'INV_DATE', 'INV_NO', 'ITEM_CODE', 'ITEM_NAME', 'QTY',
       'PRICE', 'CUSTOMER', 'SR_CODE', 'INV_TYPE', 'SELL_TYPE', 'ITEM_TYPE',
       'CUSTOMER_TYPE', 'INV_QUARTER', 'Q1', 'Q2', 'Q3', 'Q4', 'INV_MONTH',
       'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11',
       'M12', 'INV_WEEK_DAY', 'WD0', 'WD1', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6',
       'INV_MONTH_DAY'],
      dtype='object')

## **`Consolidate All Items / Pivot Data`**

In [5]:
basket = (df[df['ITEM_NAME'] != '']
          .groupby(['CUSTOMER', 'ITEM_NAME'])['QTY']
          .sum().unstack().reset_index().fillna(0)
          .set_index('CUSTOMER'))
basket

ITEM_NAME,"""Marguerite"" Invert sugar 500g.","(M-7) Alu Round Cheesecake Pan with Removeable 7"" (B359)","(M-9) Round Cake Alu Mould 9""",(No.405) 4 WHEELES FONDANT CUTTER,(SN4237) Cake Slicer 530x230 mm.,003 Cake Box 1P Silver ไม่เจาะ/กล่องเค้ก 1 ปอนด์ สีเงินสูง,003 Cake Box 1P Silver ไม่เจาะ/กล่องเค้ก 1 ปอนด์ สีเงินสูง [,016 Cake Box 3P Silver เจาะดอกไม้ /กล่องเค้ก 3 ปอนด์ เงินสูง,"01DR205, D005 Half Shape 4 cm Silicone Mould NR.15","01DR233, D033, 04DR233 Small Bordelais Silicone Mould NR.18",...,สเปรย์เนย (Bakels Sprink)(M221),สเปรย์เนย Bakels Sprink (M221),เงินประกัน,เชอร์รี่เชื่อมสีเขียว (ITALY) 500 g.(M150),เชอร์รี่เชื่อมสีแดง (ITALY) 500 g.(M149),แก้วกระดาษกาแฟเก็บอุณหภูมิ ขนาด 8 ออนซ์ + ฝา,แก๊สกระป๋อง AIKO (B057),แท่นหนีบป้ายชื่ออาหาร สีดำ 7x8.5x10 cm,แป้งข้าวโพด 1 kg (M261) (Corn Starch 1 kg.),แป้งวาฟเฟิล 1 กก./ Waffles Flour 1 kg
CUSTOMER,,,,,,,,,,,,,,,,,,,,,
0088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1STOPBAKERY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WL01316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WL01346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WP00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **`Make Dummies Data`**

In [6]:
basket_sets = basket.applymap(lambda x: 0 if x <= 0 else 1)

In [7]:
len(basket_sets.index)

4013

## **`Build up the frequent items`**

In [8]:
frequent_itemsets = apriori(basket_sets, min_support=0.005, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.005233,(10 g. White Powder Color (Titanium Dioxide))
1,0.006230,(10 g.No.3 Chocolate brown FX Macaroon color (DE186))
2,0.006728,(10 g.No.5 Erythrosine Macaroon color (DE186))
3,0.005981,(10/P GELATIN SHEETS (M116))
4,0.010466,(15x25 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s)


In [9]:
frequent_itemsets.shape

(156, 2)

## **`Create Rules`**

In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values(by=['lift', 'confidence'], ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(15x25 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s),(20x30 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s),0.010466,0.010715,0.006230,0.595238,55.550941,0.006118,2.444115
1,(20x30 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s),(15x25 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s),0.010715,0.010466,0.006230,0.581395,55.550941,0.006118,2.363887
19,(Elle&Vire Cream Cheese Plain 1.356 kg.(M153)),"(Elle&Vire Whipping Cream 35% ,1 Litre (M191))",0.012709,0.013705,0.005482,0.431373,31.474510,0.005308,1.734518
18,"(Elle&Vire Whipping Cream 35% ,1 Litre (M191))",(Elle&Vire Cream Cheese Plain 1.356 kg.(M153)),0.013705,0.012709,0.005482,0.400000,31.474510,0.005308,1.645486
2,"(Bread Flour T65 ,1 kg. แป้งขนมปัง Made in France)",(2 Kg.T55 แป้งเอนกประสงค์ Made in France M145),0.010217,0.028408,0.007974,0.780488,27.474540,0.007684,4.426143
3,(2 Kg.T55 แป้งเอนกประสงค์ Made in France M145),"(Bread Flour T65 ,1 kg. แป้งขนมปัง Made in France)",0.028408,0.010217,0.007974,0.280702,27.474540,0.007684,1.376040
20,(Service for Pick up Charge),(Spare Part for Changing or Repairing of Machine),0.009968,0.022676,0.005482,0.550000,24.254396,0.005256,2.171830
21,(Spare Part for Changing or Repairing of Machine),(Service for Pick up Charge),0.022676,0.009968,0.005482,0.241758,24.254396,0.005256,1.305695
4,(T45 แป้งขนมปัง 1 kg),(2 Kg.T55 แป้งเอนกประสงค์ Made in France M145),0.015948,0.028408,0.009718,0.609375,21.451069,0.009265,2.487276
5,(2 Kg.T55 แป้งเอนกประสงค์ Made in France M145),(T45 แป้งขนมปัง 1 kg),0.028408,0.015948,0.009718,0.342105,21.451069,0.009265,1.495759


In [11]:
rules.shape

(22, 9)

## **`Transform Final Data`**

In [12]:
# Select antecedents and consequents to new dataframe
final_rules = pd.DataFrame(rules, columns=['antecedents', 'consequents'])
# Split each antecedents and consequents for many to one
final_rules["antecedents"] = final_rules["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
final_rules["consequents"] = final_rules["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
final_rules.rename(columns={'antecedents': 'BUY', 'consequents': 'RECOMMEND'}, inplace=True)
final_rules.head()

,BUY,RECOMMEND
0,15x25 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s,20x30 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s
1,20x30 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s,15x25 cm 85 Micron ITALIAN Embrossed Vacuum Bags 100s
19,Elle&Vire Cream Cheese Plain 1.356 kg.(M153),"Elle&Vire Whipping Cream 35% ,1 Litre (M191)"
18,"Elle&Vire Whipping Cream 35% ,1 Litre (M191)",Elle&Vire Cream Cheese Plain 1.356 kg.(M153)
2,"Bread Flour T65 ,1 kg. แป้งขนมปัง Made in France",2 Kg.T55 แป้งเอนกประสงค์ Made in France M145


## **`Save Recommend Data to File`**

In [13]:
final_rules.to_csv(f'{output_path}product_recommend_2019.csv', index=False)